In [3]:
import tensorflow


In [1]:
# from keras.models import load_model
# from tensorflow.keras.preprocessing.image import img_to_array
# from tensorflow.keras.models import load_model
# classifier = load_model('ocular_model1.h5')

In [1]:
from tensorflow import keras
import os
model = keras.models.load_model('ocular_model1.h5')
# print(tensorflow.__version__)

In [4]:
import cv2 as cv



def crop_image_resize(img, ImageSize):
    #img = cv2.imread(imgPath)
    height, width = img.shape[:2]
#     print(width)
#     print(height)
    if width == height:
        return cv.resize(img, ImageSize)
        
    length = min(width, height)
    left = (width - length) // 2
    top = (height - length) // 2
    right = (width + length) // 2
    bottom = (height + length) // 2
    return cv.resize(img[top:bottom,left:right], ImageSize)

def read_image(path_of_image):
    
    img = cv.imread(path_of_image)
#     image_bw = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 
  
#     # The declaration of CLAHE  
#     # clipLimit -> Threshold for contrast limiting 
#     clahe = cv.createCLAHE(clipLimit = 5) 
#     final_img = clahe.apply(image_bw) + 30
#     print(img)
    final_image = crop_image_resize(img, (250, 250))
    cv.imwrite(path_of_image,final_image)
#     RGB_im = cv.cvtColor(final_image, cv.COLOR_BGR2RGB)
#     plt.imshow(RGB_im)
#     print(img.shape)

In [5]:
testing_image = 'ODIR-5K/Testing Images'
images = [f for f in os.listdir(testing_image) if os.path.splitext(f)[-1] == '.jpg']

for image in images:
    read_image('ODIR-5K/Testing Images/'+image)

In [6]:
import tensorflow as tf
test_ds = tf.data.Dataset.list_files("ODIR-5K/Testing Images/*.jpg", shuffle=False)

In [7]:
for f in test_ds:
    print(f.numpy())

b'ODIR-5K\\Testing Images\\937_left.jpg'
b'ODIR-5K\\Testing Images\\937_right.jpg'


In [8]:
import matplotlib.pyplot as plt

def decode_img(img):
    img = tf.io.read_file(img)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

AUTOTUNE = tf.data.experimental.AUTOTUNE
test_ds2 = test_ds.map(decode_img, num_parallel_calls=AUTOTUNE)

In [121]:
# for image in test_ds2.take(2):
#     plt.imshow(image.numpy())

In [9]:
test_ds3 = test_ds2.batch(32)
print(test_ds3)

<BatchDataset shapes: (None, None, None, 3), types: tf.float32>


In [10]:
predictions = model.predict(test_ds3)

In [11]:
import numpy
numpy.set_printoptions(precision=3,  suppress=True)
print(predictions[0])
print(predictions[1])

[0.99  0.147 0.    0.    0.029 0.003 0.    0.131]
[0.681 0.33  0.    0.    0.005 0.011 0.    0.489]


In [12]:
def merge_two(left, right):
    final_table = [0, 0, 0, 0, 0, 0, 0, 0]
    counter = 0
    not_n = 0
    for l in left:
        r = right[counter]
        if l>=r:
            final_table[counter] = l
        else:
            final_table[counter] = r
        if (l>0.5 or r >0.5) and counter>0:
            print(l ,r, counter)
            not_n=1
        counter+=1
    if not_n == 1:
        final_table[0] = 0
    return final_table

In [13]:
f_t = merge_two(predictions[0], predictions[1])

In [14]:
print(print(f_t))
print(max(f_t))
print(f_t.index(max(f_t)))
x=f_t.index(max(f_t))
if (x==0):
    print("normal")
elif (x==1):
    print("diabetes")
elif(x==2):
    print("Glaucoma")
elif(x==3):
    print("cataract")
elif(x==4):
    print("AMD")
elif(x==5):
    print("Hyper")
elif(x==6):
    print("Myopia")
elif(x==7):
    print("Other")


[0.98961127, 0.32979333, 2.2266864e-05, 4.3620666e-08, 0.029275775, 0.011309564, 1.4064368e-10, 0.4885053]
None
0.98961127
0
normal


In [15]:
numpy.set_printoptions(precision=3,  suppress=True)

print(f_t.index(max(f_t)))

0


In [16]:
init = 1000
counter = 0
for element in predictions:
    if counter %2 ==0:
        print(init, merge_two(predictions[counter], predictions[counter+1]))
        init+=1
    counter+=1

1000 [0.98961127, 0.32979333, 2.2266864e-05, 4.3620666e-08, 0.029275775, 0.011309564, 1.4064368e-10, 0.4885053]
